In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path

2023-10-04 14:22:43.685958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 14:22:45.602289: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick/miniconda3/envs/tf_env/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/nick/miniconda3/envs/tf_env/lib/
2023-10-04 14:22:45.602554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nick

In [2]:
DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/raw/master/datasets/jsb_chorales/"
FILENAME = "jsb_chorales.tgz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, 
                     cache_subdir="datasets/jsb_chorales", 
                     extract=True)

In [3]:
jsb_chorales_dir = Path(filepath).parent

In [4]:
train_files = sorted([str(file) for file in jsb_chorales_dir.glob("train/chorale_*")])
test_files = sorted([str(file) for file in jsb_chorales_dir.glob("test/chorale_*")])
val_files = sorted([str(file) for file in jsb_chorales_dir.glob("valid/chorale_*")])

In [5]:
import pandas as pd

In [6]:
train_files

['/home/nick/.keras/datasets/jsb_chorales/train/chorale_000.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_001.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_002.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_003.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_004.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_005.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_006.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_007.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_008.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_009.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_010.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_011.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_012.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_013.csv',
 '/home/nick/.keras/datasets/jsb_chorales/train/chorale_014.csv',
 '/home/ni

In [7]:
def load_chorales(filepaths):
    return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]

train_chorales = load_chorales(train_files)
test_chorales = load_chorales(test_files)
val_chorales = load_chorales(val_files)

In [8]:
notes = set()
for chorales in (train_chorales, test_chorales, val_chorales):
    for chorale in chorales:
        for chord in chorale:
            notes = notes | set(chord)
            
n_notes = len(notes)
min_note = min(notes - {0})
max_note = max(notes)

assert min_note == 36
assert max_note == 81

In [41]:
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step
    return X, Y

def preprocess(window):
    """
    Converts the values of the notes from 36-81 to 1-46
    """
    window = tf.where(window == 0, window, window - min_note + 1) # every note that is not 0 reduce by 36 + 1(for 0 note)
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows).map(preprocess)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    return dataset.prefetch(1)

In [42]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
# valid_set = bach_dataset(val_chorales)
# test_set = bach_dataset(test_chorales)

tf.Tensor(
[[74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [74 70 65 58]
 [75 70 58 55]
 [75 70 58 55]
 [75 70 60 55]
 [75 70 60 55]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]
 [77 69 62 50]
 [77 70 62 55]
 [77 70 62 55]
 [77 69 62 55]
 [77 69 62 55]
 [75 67 63 48]
 [75 67 63 48]
 [75 69 63 48]
 [75 69 63 48]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [72 69 65 53]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [74 70 65 46]
 [75 69 63 48]
 [75 69 63 48]
 [75 67 63 48]
 [75 67 63 48]
 [77 65 62 50]
 [77 65 62 50]
 [77 65 60 50]
 [77 65 60 50]
 [74 67 58 55]
 [74 67 58 55]
 [74 67 58 53]
 [74 67 58 53]
 [72 67 58 51]
 [72 67 58 51]
 [72 67 58 51]
 [72 67 58 51]
 [72 65 57 53]
 [72 65 57 53]
 [72 65 57 53]
 [72 65 57 53]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [70 65 62 46]
 [72 69 65 53]
 [72 69 65 53]

In [37]:
array = np.array([[74,0,70,65,37,48],
                  [69 ,64 ,61,45,0,0],
                  [0,76,69,64,0,81]])

In [38]:
array = tf.where(array == 0, array, array - 36 + 1)

In [39]:
array

<tf.Tensor: shape=(3, 6), dtype=int64, numpy=
array([[39,  0, 35, 30,  2, 13],
       [34, 29, 26, 10,  0,  0],
       [ 0, 41, 34, 29,  0, 46]])>

In [40]:
array = tf.reshape(array, [-1])

<tf.Tensor: shape=(18,), dtype=int64, numpy=
array([39,  0, 35, 30,  2, 13, 34, 29, 26, 10,  0,  0,  0, 41, 34, 29,  0,
       46])>

In [ ]:
keras.layers.Embedding()